Here we have some of our initial data exploration with importing the data and simple visualizations, as well as experimenting with other balanced classifying models.

In [28]:
# general imports
import numpy as np
import pandas as pd
import tensorflow as tf
import re
import string
from collections import Counter

# tensorflow model imports
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow import keras

# sklearn modeling imports
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.utils import class_weight

# visualization imports
import seaborn as sns
from matplotlib import pyplot as plt
import plotly.express as px

In [2]:
# importing data from drive file
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/My\ Drive/Colab\ Notebooks

/content/drive/My Drive/Colab Notebooks


In [4]:
import functs

In [5]:
path='/content/drive/My Drive/Colab Notebooks/Loan Data DEC22-MAR23.xlsx' # importing our data
mar23 = pd.read_excel(path,'MAR23') # importing the first, march 2023 excel sheet
dec22 = pd.read_excel(path,'DEC22') # importing the second, dec 2022 excel sheet

We started by exploring our data and investigating relationships between variables.

In [7]:
dec22.shape, mar23.shape

((85310, 24), (95768, 24))

In [8]:
dec22.head()

,cutoff_date,loan_id,loan_group,loan_program,orig_credit_score,loan_purpose_type,property_type,property_state,lien_position,original_cltv,...,orig_date,original_loan_term,rterm,original_interest_rate,current_interest_rate,delinquent_dt,days_delinquent,loan_status,original_dti_rate,charge off?
0,2022-12-30,loan_10000057796,NaN,PRIMARY,712,Debt consolidation,SFR,FL,Junior,0.7644,...,2022-02-15,120,110,0.0670,0.0670,NaT,0,IN_REPAY,0.2661,No
1,2022-12-30,loan_10000046566,NaN,PRIMARY,718,Debt consolidation,TOWNHOUSE/ROWHOUSE,MT,Junior,0.8996,...,2022-04-15,360,352,0.0820,0.0820,NaT,0,IN_REPAY,0.2869,No
2,2022-12-30,loan_10000041028,NaN,PRIMARY,650,Debt consolidation,CONDO,AZ,Junior,0.4856,...,2021-07-15,120,103,0.0700,0.0700,NaT,0,IN_REPAY,0.4910,No
3,2022-12-30,loan_10000067474,NaN,PRIMARY,680,Home improvement,SFR,FL,Junior,0.6394,...,2021-10-15,180,166,0.0664,0.0664,NaT,0,IN_REPAY,0.4993,No
4,2022-12-30,loan_10000047479,loan_10000092177,DRAW,770,Debt consolidation,SFR,IL,Junior,0.9000,...,2022-11-10,105,104,0.1030,0.1030,NaT,0,IN_REPAY,0.2032,No


# Initial Visualizations

In [9]:
#create function to create new column categorizing credit score
def categorize_credit_score(df,credit_column):
  """
  This function takes a dataframe input, as well as a column name for categorizing
  credit score in a meaningful way
  """
  conditions =[(df[credit_column]>=800),
            (df[credit_column]<800) &(df[credit_column]>=740),
            (df[credit_column]<740) &(df[credit_column]>=670),
            (df[credit_column]<670) &(df[credit_column]>=580),
            (df[credit_column]<580) &(df[credit_column]>=300)]
  values=['Excellent','Very good','Good','Fair','Poor']
  df['credit_score_category']=np.select(conditions, values)

In [10]:
# we then applied this function to both our march and dec df's
categorize_credit_score(dec22,'orig_credit_score')
categorize_credit_score(mar23,'orig_credit_score')
dec22.head()

,cutoff_date,loan_id,loan_group,loan_program,orig_credit_score,loan_purpose_type,property_type,property_state,lien_position,original_cltv,...,original_loan_term,rterm,original_interest_rate,current_interest_rate,delinquent_dt,days_delinquent,loan_status,original_dti_rate,charge off?,credit_score_category
0,2022-12-30,loan_10000057796,NaN,PRIMARY,712,Debt consolidation,SFR,FL,Junior,0.7644,...,120,110,0.0670,0.0670,NaT,0,IN_REPAY,0.2661,No,Good
1,2022-12-30,loan_10000046566,NaN,PRIMARY,718,Debt consolidation,TOWNHOUSE/ROWHOUSE,MT,Junior,0.8996,...,360,352,0.0820,0.0820,NaT,0,IN_REPAY,0.2869,No,Good
2,2022-12-30,loan_10000041028,NaN,PRIMARY,650,Debt consolidation,CONDO,AZ,Junior,0.4856,...,120,103,0.0700,0.0700,NaT,0,IN_REPAY,0.4910,No,Fair
3,2022-12-30,loan_10000067474,NaN,PRIMARY,680,Home improvement,SFR,FL,Junior,0.6394,...,180,166,0.0664,0.0664,NaT,0,IN_REPAY,0.4993,No,Good
4,2022-12-30,loan_10000047479,loan_10000092177,DRAW,770,Debt consolidation,SFR,IL,Junior,0.9000,...,105,104,0.1030,0.1030,NaT,0,IN_REPAY,0.2032,No,Very good


In [12]:
#sort df alphabetically for column
def sort_alph(df,column_name):
  """
  this is a simple function that takes a dataframe and column name input and
  sorts the dataframe by the given column, returning a sorted df
  """
  return df.sort_values(by=[column_name])

dec22_sorted=sort_alph(dec22,'loan_status')
mar23_sorted=sort_alph(mar23,'loan_status')

In [13]:
credit_order=dict(credit_score_category=["Excellent","Very good","Good","Fair","Poor"])

The following histograms compare the loan's credit score classification and the distribution of loan status accordingly. They are also all normalized to visually see the differences in loan class more clearly, since there are unequal counts of loans in each credit score bin.

In [38]:
fig = px.histogram(dec22_sorted,
                   x = "credit_score_category",
                   color="loan_status",
                   category_orders=credit_order,
                   barnorm="percent",
                   title ="December 2022 Credit Score Loan Distributions")
fig.show()

In [39]:
fig = px.histogram(mar23_sorted,
                   x = "credit_score_category",
                   color="loan_status",
                   category_orders=credit_order,
                   barnorm="percent",
                   title ="March 2023 Credit Score Loan Distributions")
fig.show()

One of the highest correlations that we saw between loan status and the other feature columns was the credit score. As we can see in both graphs, there are far more loans classified as delinquent, the labels at the bottom of the bars, as the graph moves toward the right, with lower credit scores. This confirms the correlation between credit score and delinquency, and provides interesting implications for our future analysis of how to predict delinquent loans.

# Balanced Weights Model

The balanced weights model is another commonly used resampling method that combines the approaches of over and undersampling, and instead weighs each class individually.

First we will clean our data, following the same code as in our main notebook.

In [25]:
functs.combine_delq(mar23)
functs.combine_delq(dec22)

import sqlite3
conn=sqlite3.connect("loans.db") # creates a local sql database connection
cursor=conn.cursor()
mar23.to_sql('mar23', conn, if_exists='replace', index=False) # creates mar23 sql table
dec22.to_sql('dec22',conn,if_exists='replace',index=False) # creates dec22 sql table

# our sql query command for merging the two dataframes with the same loan ID's
cmd=\
"""
SELECT mar23.*, dec22.delq AS decDelq
FROM mar23 LEFT JOIN dec22 ON mar23.loan_id = dec22.loan_id
"""
loan_merged=pd.read_sql_query(cmd,conn)
conn.close()

functs.merge_delq_columns(loan_merged)

functs.clean_data_for_ml(loan_merged)
loan_merged = loan_merged.dropna()

In [26]:
# the distribution of our class labels
print(loan_merged['Delq'].value_counts(normalize = True))
print(loan_merged["Delq"].value_counts())

1    0.967134
0    0.032866
Name: Delq, dtype: float64
1    92282
0     3136
Name: Delq, dtype: int64


First going to split the data and then train it on a baseline method, with no weights. We will later apply the weights and determine its effectiveness.

In [29]:
# first going to split the data
X = loan_merged.drop('Delq', axis=1).values
y = loan_merged['Delq'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# prints the number of data points in each train and test, and then how many of each category
print(f"Total X_train: {X_train.shape[0]}")
print(f"Total X_test: {X_test.shape[0]}")
print(f"The training dataset has {sorted(Counter(y_train).items())[0][1]} delq values and {sorted(Counter(y_train).items())[1][1]} non-delq values")

Total X_train: 76334
Total X_test: 19084
The training dataset has 2528 delq values and 73806 non-delq values


Here, we are using the same keras sequential model that we trained our over and under sampled data in our main notebook.

In [30]:
model_bl = tf.keras.Sequential([
    layers.Dense(18, activation='relu'),
    layers.Dense(10, activation='relu'),
    layers.Dense(5, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model_bl.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=[tf.keras.metrics.Precision(),
                        tf.keras.metrics.Recall()]
)
history_bl=model_bl.fit(
        X_train,
        y_train,
        epochs=20,
        batch_size=256,
        validation_data=(X_test, y_test)
)

Epoch 1/20
299/299 [==============================] - 4s 6ms/step - loss: 78.4922 - precision: 0.9686 - recall: 0.9357 - val_loss: 0.5776 - val_precision: 0.9691 - val_recall: 0.9916
Epoch 2/20
299/299 [==============================] - 1s 4ms/step - loss: 0.5377 - precision: 0.9679 - recall: 0.9937 - val_loss: 0.4292 - val_precision: 0.9690 - val_recall: 0.9964
Epoch 3/20
299/299 [==============================] - 1s 4ms/step - loss: 0.4077 - precision: 0.9675 - recall: 0.9962 - val_loss: 0.3553 - val_precision: 0.9694 - val_recall: 0.9939
Epoch 4/20
299/299 [==============================] - 1s 3ms/step - loss: 0.3475 - precision: 0.9676 - recall: 0.9965 - val_loss: 0.3105 - val_precision: 0.9682 - val_recall: 0.9998
Epoch 5/20
299/299 [==============================] - 1s 3ms/step - loss: 0.2752 - precision: 0.9675 - recall: 0.9976 - val_loss: 0.2465 - val_precision: 0.9681 - val_recall: 0.9995
Epoch 6/20
299/299 [==============================] - 1s 3ms/step - loss: 0.2287 - precis

In [31]:
# Prediction
bl_model_prediction = model_bl.predict(X_test)
bl_model_classes =  [1 if i>0.5 else 0 for i in bl_model_prediction]

# Check the model performance
print(classification_report(y_test, bl_model_classes))

597/597 [==============================] - 2s 3ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       608
           1       0.97      1.00      0.98     18476

    accuracy                           0.97     19084
   macro avg       0.48      0.50      0.49     19084
weighted avg       0.94      0.97      0.95     19084



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



These were our unbalanced dataset model results, and as we can see the model did not even train on our delinquent loans since they make up such a small portion of our overall dataset. We can see that we get incredibly high precision and recall values for the non delinquent loans, but 0% of the delinquent loans were accurately classified. This is what motivated us for using a resampling method in the first place.
Next, we will apply the balanced weights method.

Our balancing compution is defined by the following formula:

n_samples / (n_classes * np.bincount(y))

In [35]:
sklearn_weights = class_weight.compute_class_weight('balanced', classes = np.unique(y_train), y = y_train)
sklearn_weights = dict(enumerate(sklearn_weights))
sklearn_weights

{0: 15.097705696202532, 1: 0.5171259789177032}

The printed returns above are the calculated weights, where the delinquent class is given a much higher weight of about 15, while the larger delinquent class is given a weight of 0.5

In [36]:
model_bal = tf.keras.Sequential([
    layers.Dense(18, activation='relu'),
    layers.Dense(10, activation='relu'),
    layers.Dense(5, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model_bal.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=["accuracy"]
)
history_bal=model_bal.fit(
        X_train,
        y_train,
        epochs=10,
        batch_size=256,
        class_weight = sklearn_weights, # here we add our weights to the fit function
        validation_data=(X_test, y_test)
)

Epoch 1/10
299/299 [==============================] - 3s 5ms/step - loss: 84.8041 - accuracy: 0.7194 - val_loss: 17.6146 - val_accuracy: 0.8191
Epoch 2/10
299/299 [==============================] - 1s 3ms/step - loss: 35.3386 - accuracy: 0.7277 - val_loss: 8.0251 - val_accuracy: 0.9277
Epoch 3/10
299/299 [==============================] - 1s 3ms/step - loss: 25.5495 - accuracy: 0.7425 - val_loss: 3.5825 - val_accuracy: 0.8636
Epoch 4/10
299/299 [==============================] - 1s 3ms/step - loss: 20.9378 - accuracy: 0.7234 - val_loss: 2.5282 - val_accuracy: 0.8873
Epoch 5/10
299/299 [==============================] - 1s 3ms/step - loss: 17.6806 - accuracy: 0.7446 - val_loss: 4.5159 - val_accuracy: 0.8527
Epoch 6/10
299/299 [==============================] - 1s 3ms/step - loss: 18.8680 - accuracy: 0.7308 - val_loss: 3.0724 - val_accuracy: 0.8979
Epoch 7/10
299/299 [==============================] - 1s 3ms/step - loss: 12.7126 - accuracy: 0.7377 - val_loss: 3.0753 - val_accuracy: 0.884

In [37]:
# Prediction
bal_model_prediction = model_bal.predict(X_test)
bal_model_classes =  [1 if i>0.5 else 0 for i in bal_model_prediction]

# Check the model performance
print(classification_report(y_test, bal_model_classes))

597/597 [==============================] - 1s 2ms/step
              precision    recall  f1-score   support

           0       0.15      0.31      0.20       608
           1       0.98      0.94      0.96     18476

    accuracy                           0.92     19084
   macro avg       0.56      0.63      0.58     19084
weighted avg       0.95      0.92      0.93     19084



Now evaluating these scores, they are significantly higher than our baseline model without resampling, however we are still struggling with very low precision and recall scores for the delinquent loans. This was one of our early resampling model attempts, and after experimenting with over and undersampling, we found that those approaches performed much better for our given data.